In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 95.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 8.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
import random
import numpy as np

import spacy 

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, Dataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import BertTokenizer,BertModel, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from collections import defaultdict

from sklearn.metrics import confusion_matrix, classification_report,precision_score,accuracy_score,f1_score

import logging 
logging.basicConfig(level = logging.ERROR)



In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from datasets import load_dataset

dataset = load_dataset("onestop_english")

Generating train split:   0%|          | 0/567 [00:00<?, ? examples/s]

Dataset onestop_english downloaded and prepared to /root/.cache/huggingface/datasets/onestop_english/default/1.1.0/6b19eec5680862ad1cf1990e98b06a98d1fa4c85f3585dc4dfab93f52b89d9cf. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 567
    })
})

In [7]:
import pandas as pd

df = pd.DataFrame(dataset['train'])
df.columns = ['content', 'labels']


In [8]:
df

,content,labels
0,"When you see the word Amazon, what’s the first...",0
1,"To tourists, Amsterdam still seems very libera...",0
2,"Anitta, a music star from Brazil, has millions...",0
3,Google has made maps of the world’s highest mo...,0
4,The auction of a Banksy painting that disappea...,0
...,...,...
562,"In typical bad-boyfriend style, Dan Sullivan w...",2
563,Thousands of people protested on Australia’s b...,2
564,1 Race engineer \nA race engineer liaises betw...,2
565,More than one million British workers might be...,2


In [9]:
df.labels.value_counts()


0    189
1    189
2    189
Name: labels, dtype: int64

In [10]:
num_diff = df.labels.unique()
diff = ['beginner','intermediate','advanced']

label = dict(zip(num_diff, diff))

diff = list(diff)


In [11]:
df['Difficulty'] = df.labels.replace(label)
df.head()



,content,labels,Difficulty
0,"When you see the word Amazon, what’s the first...",0,beginner
1,"To tourists, Amsterdam still seems very libera...",0,beginner
2,"Anitta, a music star from Brazil, has millions...",0,beginner
3,Google has made maps of the world’s highest mo...,0,beginner
4,The auction of a Banksy painting that disappea...,0,beginner


Data Preprocessing

In [12]:
import re

In [13]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [13]:

def clean_text(text):
    #text=' '.join([contraction_mapping[i] if i in contraction_mapping.keys() else i for i in text.split()])
    text=re.sub("'s","",text)
    if text.split()[0] == 'Intermediate':
      text = ' '.join(text.split()[1:])
    text=' '.join([i for i in text.split() if i.isalpha()])
    text=re.sub('[^a-zA-Z]'," ",text)

    
    return text

In [14]:
df['content'] = df['content'].apply(clean_text)


In [15]:
df[df['Difficulty']=='intermediate']


,content,labels,Difficulty
189,When you see the word whats the first thing yo...,1,intermediate
190,To Amsterdam still seems very Recently the cit...,1,intermediate
191,Brazils latest funk has won millions of fans b...,1,intermediate
192,It has mapped the worlds highest the ocean the...,1,intermediate
193,The controversial auction of a Banksy mural th...,1,intermediate
...,...,...,...
373,In typical Dan Sullivan was late to breakfast ...,1,intermediate
374,Thousands of people protested on Australias be...,1,intermediate
375,Race engineer A race engineer liaises between ...,1,intermediate
376,More than one million British workers might be...,1,intermediate


In [16]:
df

,content,labels,Difficulty
0,When you see the word the first thing you thin...,0,beginner
1,To Amsterdam still seems very Recently the May...,0,beginner
2,a music star from has millions of but she is a...,0,beginner
3,Google has made maps of the highest the ocean ...,0,beginner
4,The auction of a Banksy painting that disappea...,0,beginner
...,...,...,...
562,In typical Dan Sullivan was late to breakfast ...,2,advanced
563,Thousands of people protested on beaches again...,2,advanced
564,Race engineer A race engineer liaises between ...,2,advanced
565,More than one million British workers might be...,2,advanced


mean tree height

In [17]:
en_nlp = spacy.load('en_core_web_sm')

In [18]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [19]:
def walk_tree(node, depth):
    if node.n_lefts + node.n_rights > 0:
        return max(walk_tree(child, depth + 1) for child in node.children)
    else:
        return depth

def get_mean_tree_height(text):
  depths = []
  sentences = sent_tokenize(text)
  for sentence in sentences:
    doc = en_nlp(sentence)
    depth = [walk_tree(sent.root, 0) for sent in doc.sents]
    depths.append(depth)
  return np.mean(depths)

In [21]:
df['content'][0]

'When you see the word the first thing you think of the biggest the longest river or the largest internet shop and which do you think is most These are the questions in a debate about the Brazil and Peru have made objections to a bid made by the US online shop for the domain Amazon has asked for its company name to be a domain name but the South American governments say this would stop the use of this internet address for environmental indigenous rights and other public interest There are many other disputed claims to including Until the differences between governmental and other types of identity were easy to see in every internet address by the use of and other But soon there are going to be more of these categories or generic domains as they are technically The Internet Corporation for Assigned Names and Numbers has had bids worth almost for hundreds of new gTLDs to add to the that we use Amazon has applied for many new including and But the one that has caused most discussion is it

In [22]:
get_mean_tree_height(df['content'][0])

11.0

In [23]:
get_mean_tree_height(df['content'][471])

19.0

In [19]:
sen_len = []
for i in df['content']:
  curr_sen_len = []
  for j in sent_tokenize(i):
    curr_sen_len.append(len(word_tokenize(j)))
  mean_len = np.mean(curr_sen_len)
  sen_len.append(mean_len)

In [20]:
sen_len[400:410]

[864.0, 661.0, 686.0, 830.0, 671.0, 665.0, 787.0, 842.0, 565.0, 733.0]

In [24]:
df['mean_tree_height'] = df['content'].apply(get_mean_tree_height)

In [22]:
df

,content,labels,Difficulty,mean_sentence_len
0,When you see the word the first thing you thin...,0,beginner,351.0
1,To Amsterdam still seems very Recently the May...,0,beginner,380.0
2,a music star from has millions of but she is a...,0,beginner,387.0
3,Google has made maps of the highest the ocean ...,0,beginner,432.0
4,The auction of a Banksy painting that disappea...,0,beginner,371.0
...,...,...,...,...
562,In typical Dan Sullivan was late to breakfast ...,2,advanced,445.0
563,Thousands of people protested on beaches again...,2,advanced,516.0
564,Race engineer A race engineer liaises between ...,2,advanced,704.0
565,More than one million British workers might be...,2,advanced,684.0


In [25]:
df

,content,labels,Difficulty,mean_tree_height
0,When you see the word the first thing you thin...,0,beginner,11.000000
1,To Amsterdam still seems very Recently the May...,0,beginner,13.000000
2,a music star from has millions of but she is a...,0,beginner,9.571429
3,Google has made maps of the highest the ocean ...,0,beginner,18.000000
4,The auction of a Banksy painting that disappea...,0,beginner,16.000000
...,...,...,...,...
562,In typical Dan Sullivan was late to breakfast ...,2,advanced,10.000000
563,Thousands of people protested on beaches again...,2,advanced,12.000000
564,Race engineer A race engineer liaises between ...,2,advanced,17.000000
565,More than one million British workers might be...,2,advanced,14.750000


In [32]:
np.mean(df[df['Difficulty']=='beginner']['mean_tree_height'])

11.44796504201266

In [31]:
np.mean(df[df['Difficulty']=='intermediate']['mean_tree_height'])

11.52747402509307

In [30]:
np.mean(df[df['Difficulty']=='advanced']['mean_tree_height'])

11.44796504201266

In [33]:
from transformers import BertTokenizer,BertModel, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup


In [34]:
class DiffDataset(Dataset):
    def __init__(self, contents, labels, features, tokenizer, max_len):
        self.contents = contents
        self.labels = labels
        self.tokenizer = tokenizer
        self.features = features
        self.max_len = max_len
  
    def __len__(self):
        return len(self.contents)
  
    def __getitem__(self, item):
        content = str(self.contents[item])
        labels = self.labels[item]
        features = self.features[item]
        # Tokenizing the texts, while also including special tokens 
        # for start and end of the text, as well as padding
        encoding = self.tokenizer.encode_plus(
          content,
          add_special_tokens=True,
          max_length=self.max_len,
          return_token_type_ids=False,
          pad_to_max_length=True,
          return_attention_mask=True,
          return_tensors='pt', # We return here the data as Pytorch Tensor
        )

        return {
          'content': content,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'labels': torch.tensor(labels, dtype=torch.long), 
          'features': torch.tensor([features], dtype=torch.long)
        }


In [25]:
torch.tensor([[12, 23, 45]], dtype=torch.long).shape

torch.Size([1, 3])

In [35]:
df_temp, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_temp, test_size=0.1, random_state=42)

print("size of training set : " + str(df_train.shape[0]))
print("size of validation set : " + str(df_val.shape[0]))
print("size of test set : " + str(df_test.shape[0]))



size of training set : 407
size of validation set : 46
size of test set : 114


In [40]:
def generate_dataloader(df, tokenizer, max_len, batch_size):
    ds = DiffDataset(
        contents=df.content.to_numpy(),
        labels=df.labels.to_numpy(),
        features = df.mean_tree_height.to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len,
      )
    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=2
      )


In [37]:
#model_name = 'bert-base-multilingual-uncased'
model_name = 'bert-base-uncased'

In [38]:
tokenizer = BertTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [41]:
max_len = 256 
batch_size = 8

train_dataloader = generate_dataloader(df_train, tokenizer, max_len,batch_size)
val_dataloader = generate_dataloader(df_val, tokenizer, max_len,batch_size)
test_dataloader = generate_dataloader(df_test, tokenizer, max_len,batch_size)

data = next(iter(train_dataloader))



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [42]:
class DiffClassifier(nn.Module):

    def __init__(self, n_classes):
        super(DiffClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        # Adding drop out, keeping 90% of the last neurons of the raw BERT model
        #self.drop = nn.Dropout(p=0.1)
        # The last linear layer for multiclass classification
        self.out = nn.Linear(self.bert.config.hidden_size+1, n_classes)
  
    # Forward propagation function
    def forward(self, input_ids, attention_mask, features):
        model_outs = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask
        )
        last_hidden_state = model_outs[0]
        pooled_output = model_outs[1]

        x = torch.cat((pooled_output, features) , dim=1)

        return self.out(x)


In [43]:
len(diff)

3

In [44]:
model = DiffClassifier(len(diff))

# Running the classifier on GPU
model = model.to(device)


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [45]:
input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)
features = data['features'].to(device)
print(input_ids.shape)
print(attention_mask.shape)
print(features.shape)

torch.Size([8, 256])
torch.Size([8, 256])
torch.Size([8, 1])


In [46]:
features

tensor([[ 8],
        [ 7],
        [11],
        [12],
        [17],
        [14],
        [16],
        [14]], device='cuda:0')

In [47]:
optimizer = AdamW(model.parameters(),
                  lr=2e-5, 
                  weight_decay = 0.2,
                  correct_bias=False)
epochs = 5
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0.1,
                                            num_training_steps=len(train_dataloader)*epochs)
loss_fn = nn.CrossEntropyLoss().to(device)


In [48]:
# Computing multiclass precision for the outputs of the model
def compute_precision(outputs, labels):
  op = outputs.cpu()
  la = labels.cpu()
  _, preds = torch.max(op, dim=1)
  # We choose 'weighted' averaging of the precision of each label because it takes into account the imbalance of labels in our tweets dataset
  # other viable averaging methods are 'micro'
  return torch.tensor(precision_score(la, preds, average='weighted',zero_division=0))

In [49]:
def train(model, train_dataloader,optimizer,scheduler):

        # Reset tracking variables at the beginning of each epoch
        precision, correct_predictions, batch_counts = 0, 0, 0
        losses = []

        # Put the model into the training mode
        model.train()

  
        # For each batch of training data...
        for d in train_dataloader:
              batch_counts +=1
              # Load batch to GPU
              input_ids = d["input_ids"].to(device)
              attention_mask = d["attention_mask"].to(device)
              labels = d["labels"].to(device)
              features = d["features"].to(device)

              outputs = model(input_ids=input_ids,attention_mask=attention_mask, features=features)
              
              _, preds = torch.max(outputs, dim=1)
              
              # Compute loss and accumulate the loss values

              loss = loss_fn(outputs, labels)

              correct_predictions += torch.sum(preds == labels)
              losses.append(loss.item())
              precision +=  compute_precision(outputs, labels)
            
              loss.backward()

              # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
              torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
              # Update parameters and the learning rate  
              optimizer.step()
              scheduler.step()
              optimizer.zero_grad()
          # Accuracy, loss, precision
        return correct_predictions.double() / len(df_train), np.mean(losses), precision/batch_counts

In [50]:
def eval(model, valid_dataloader, loss_fn, device, n):
    model = model.eval()


    correct_predictions , precision ,batch_counts = 0,0,0
    losses = []

    with torch.no_grad():
        for d in valid_dataloader:
            batch_counts += 1

            # Preparing inputs
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["labels"].to(device)
            features = d["features"].to(device)

            # Running inference using the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                features=features
              )
            
            # Running softmax on the outputs
            _, preds = torch.max(outputs, dim=1)

            # Computing loss function
            loss = loss_fn(outputs, labels)

            # Counting the correct occurences
            correct_predictions += torch.sum(preds == labels)

            # Computing the precision (true positives/true positives + false positives) 
            # for each class and label, and find their average weighted by support 
            precision += compute_precision(outputs,labels)

            losses.append(loss.item())
    # Accuracy, loss, precision 
    return correct_predictions.double()/n, np.mean(losses), precision/batch_counts 

In [51]:
history = defaultdict(list)
b_accuracy = 0

for epoch in range(epochs):
  
  print(f'Epoch {epoch + 1}/{epochs}')
  print('-' * 20)

  train_acc, train_loss, train_preci = train(
        model,
        train_dataloader,    
        optimizer, 
        scheduler,
      )
  print(f"Train : Loss {train_loss}, Accuracy : {train_acc*100:.2f} %, Precision : {train_preci}")

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['train_precision'].append(train_preci)

  val_acc, val_loss, val_preci = eval(
        model,
        val_dataloader,
        loss_fn, 
        device,
        len(df_val),
      )

  print(f'Val : Loss :{val_loss}, Accuracy : {val_acc*100:.2f} %, Precision : {val_preci}')  
  print()

  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)
  history['val_precision'].append(val_preci)
  
  if val_acc > b_accuracy:
    torch.save(model.state_dict(), 'bert_model_baseline.bin')
    b_accuracy = val_acc

Epoch 1/5
--------------------


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Train : Loss 0.9388993622041216, Accuracy : 52.33 %, Precision : 0.4204986161131119


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Val : Loss :0.5631325493256251, Accuracy : 84.78 %, Precision : 0.8467592592592593

Epoch 2/5
--------------------


...

Val : Loss :0.3220052714459598, Accuracy : 91.30 %, Precision : 0.9027777777777778

Epoch 5/5
--------------------


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Train : Loss 0.16117118141047804, Accuracy : 94.84 %, Precision : 0.9640114379084967


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Val : Loss :0.3301048940047622, Accuracy : 91.30 %, Precision : 0.9027777777777778



In [52]:
history['train_acc']

[tensor(0.5233, device='cuda:0', dtype=torch.float64),
 tensor(0.7985, device='cuda:0', dtype=torch.float64),
 tensor(0.9091, device='cuda:0', dtype=torch.float64),
 tensor(0.9386, device='cuda:0', dtype=torch.float64),
 tensor(0.9484, device='cuda:0', dtype=torch.float64)]

In [53]:
history['val_acc']

[tensor(0.8478, device='cuda:0', dtype=torch.float64),
 tensor(0.8478, device='cuda:0', dtype=torch.float64),
 tensor(0.7826, device='cuda:0', dtype=torch.float64),
 tensor(0.9130, device='cuda:0', dtype=torch.float64),
 tensor(0.9130, device='cuda:0', dtype=torch.float64)]

In [54]:
def testing(model, dataloader):
  model = model.eval()
  
  contents = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in dataloader:

      texts = d["content"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      labels = d["labels"].to(device)
      features = d["features"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask, 
        features=features
      )
      _, preds = torch.max(outputs, dim=1)

      probs = F.softmax(outputs, dim=1)

      contents.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(probs)
      real_values.extend(labels)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return contents, predictions, prediction_probs, real_values

In [55]:
y_contents, y_pred, y_pred_probs, y_test = testing(
  model,
  test_dataloader
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [56]:
print(classification_report(y_test, y_pred, target_names=diff))

              precision    recall  f1-score   support

    beginner       0.95      0.91      0.93        43
intermediate       0.92      0.89      0.91        38
    advanced       0.86      0.94      0.90        33

    accuracy                           0.91       114
   macro avg       0.91      0.91      0.91       114
weighted avg       0.91      0.91      0.91       114



In [46]:
y_pred_probs

tensor([[2.3837e-01, 5.0683e-04, 7.6112e-01],
        [4.1651e-01, 2.8099e-01, 3.0250e-01],
        [3.8738e-01, 1.0506e-03, 6.1156e-01],
        [2.5648e-01, 7.1417e-05, 7.4345e-01],
        [4.4118e-01, 2.2726e-02, 5.3609e-01],
        [4.3408e-01, 1.7285e-05, 5.6590e-01],
        [6.9258e-01, 6.2886e-03, 3.0113e-01],
        [7.8952e-02, 1.0096e-06, 9.2105e-01],
        [9.7950e-02, 4.7561e-06, 9.0205e-01],
        [4.9245e-01, 4.9232e-04, 5.0706e-01],
        [8.5769e-02, 9.0988e-01, 4.3558e-03],
        [7.8143e-01, 1.7404e-02, 2.0116e-01],
        [5.2828e-01, 2.5395e-01, 2.1777e-01],
        [5.6318e-01, 1.1356e-04, 4.3670e-01],
        [7.9430e-02, 1.3755e-06, 9.2057e-01],
        [1.2785e-01, 8.6720e-01, 4.9415e-03],
        [7.0874e-01, 1.8307e-02, 2.7296e-01],
        [5.5545e-01, 4.3870e-03, 4.4016e-01],
        [6.6026e-02, 1.5989e-07, 9.3397e-01],
        [2.1393e-01, 7.7026e-01, 1.5810e-02],
        [6.4813e-01, 2.9735e-01, 5.4516e-02],
        [7.2737e-01, 2.1102e-01, 6

In [47]:
out_df = pd.DataFrame()
out_df['content'] = y_contents
out_df['true_label'] = y_test
out_df['pred_label'] = y_pred

In [48]:
out_df

,content,true_label,pred_label
0,What is it like to look at the very last of To...,2,2
1,SeaWorld has suffered an collapse in profits a...,2,0
2,On the market square in Rjukan stands a statue...,1,2
3,The world shares him and London claims him but...,2,2
4,The regulation eight hours in the office is Th...,2,2
...,...,...,...
109,Some cities have Lima has black They fly in gr...,0,1
110,David Cameron has declared a in the Scottish i...,2,2
111,Clay Cockrell is sitting in his of ce across t...,1,0
112,Police and intelligence agencies around the wo...,0,1


In [49]:
out_df.to_csv('out_BERT_256_simple features.csv')